# L03 Kata - Multilabel classification

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *

### Exploring the data

In [ ]:
path = Config.data_path()/'planet_sample'
planet = untar_data(URLs.PLANET_SAMPLE)
path.ls()

In [ ]:
df = pd.read_csv(path/'labels.csv')
df.head()

### Loading the data

In [ ]:
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)


In [ ]:
np.random.seed(42)
src = (ImageList.from_csv(path, 'labels.csv', folder='train', suffix='.jpg')
       .split_by_rand_pct(0.2)
       .label_from_df(label_delim=' '))


In [ ]:
data = (src.transform(tfms, size=128)
        .databunch().normalize(imagenet_stats))


In [ ]:
data.show_batch(rows=3, figsize=(12,9))

### Training

In [ ]:
arch = models.resnet50

In [ ]:
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)
learn = cnn_learner(data, arch, metrics=[acc_02, f_score])

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
lr = 0.01

In [ ]:
learn.fit_one_cycle(5, slice(lr))

In [ ]:
learn.save('stage-1-rn50')

### Fine tune model

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

In [ ]:
learn.save('stage-2-rn50')

### Accuracy analysis

### Predict on CPU